In [7]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 1. Veri Setini Yükleme
# Görüntülerinizin bulunduğu ana klasörü belirtin
#main_folder = 'Data/train'
#C:\Users\furka\PycharmProjects\Celebrity-Face-Detection\Data\train
main_folder = 'C:/Users/furka/PycharmProjects/Celebrity-Face-Detection/Data/train'
# Her bir etiketin bulunduğu alt klasörleri alın
labels = [label for label in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, label))]

image_paths = []
for label in labels:
    label_folder = os.path.join(main_folder, label)
    for image_name in os.listdir(label_folder):
        image_path = os.path.join(label_folder, image_name)
        image_paths.append((image_path, label))

# 2. Görüntü Boyutlarını Düzenleme
# Tüm görüntüleri aynı boyuta getirerek homojen bir veri seti oluşturun.
# Örneğin, 64x64 boyutuna düzenleyelim.
image_size = (64, 64)
images = []

for path, label in image_paths:
    image = cv2.imread(path)
    image = cv2.resize(image, image_size)
    images.append(image)

# 3. Etiketleri Sayısal Formata Çevirme
# Etiketleri sayısal formata çevirerek model eğitimi için hazırlık yapın.
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform([label for _, label in image_paths])

# 4. Veri Setini Train ve Test Olarak Ayırma
X_train, X_test, y_train, y_test = train_test_split(images, encoded_labels, test_size=0.2, random_state=42)

# 5. Görüntüleri Normalleştirme (Opsiyonel)
# Görüntü piksellerini 0 ile 1 arasında bir değere normalleştirme işlemi yapabilirsiniz.
X_train = np.array(X_train) / 255.0
X_test = np.array(X_test) / 255.0

# Şimdi veri setiniz önişleme adımlarından geçti ve model eğitimi için hazır hale geldi.

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)
#print lahels 
print(labels)

X_train shape: (1142, 64, 64, 3)
X_test shape: (286, 64, 64, 3)
y_train shape: (1142,)
y_test shape: (286,)
['Jake-Big-Face', 'Jake-Small-Face', 'Jake-Small-Face Jimmy-Small-Face', 'Jake-Small-Face Jimmy-Small-Face Other', 'Jake-Small-Face Other', 'Jimmy-Big-Face']


In [9]:
import tensorflow as tf
from tensorflow.keras import layers, models

# CNN Modeli Oluşturma
model = models.Sequential()

# Evrişim Katmanları
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

# Düzleştirme ve Yoğun Katmanlar
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(5, activation='softmax'))  # Çıkış katmanı, sınıf sayısına uygun olmalı (5 etiket)

# Modeli Derleme
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Modeli Eğitme
model.fit(X_train, y_train, epochs=30, validation_data=(X_test, y_test))

# Modeli Değerlendirme
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('\nTest accuracy:', test_acc)
# Modeli "my_model.h5" adlı bir dosyaya kaydetme
model.save("my_model.h5")

# Modeli yüklemek için kullanılabilecek kod



ValueError: object __array__ method not producing an array

In [13]:
# loaded_model = tf.keras.models.load_model("my_model.h5")
import cv2
from mtcnn.mtcnn import MTCNN

from mtcnn.mtcnn import MTCNN

# Video dosyasını oku
video_path = 'Assignment/Training_Video_Jimmy Kimmel-Jake Johnson.mp4'
cap = cv2.VideoCapture(video_path)

# MTCNN yüz algılama modelini yükle
detector = MTCNN()




1/1 [==============================] - 0s 23ms/step


KeyboardInterrupt: 

In [14]:

import cv2
import numpy as np

# Eğitilmiş modeli yükleme
# Örneğin: model = tf.keras.models.load_model('your_trained_model.h5')

# Video dosyasını açma
video_path = 'Assignment/Training_Video_Jimmy Kimmel-Jake Johnson.mp4'  # Video dosyasının yolu
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Görüntüyü modele uygun formata getirme
    input_frame = cv2.resize(frame, (64, 64))
    input_frame = np.expand_dims(input_frame, axis=0)

    # Modeli kullanarak tahminde bulunma 
    predictions = model.predict(input_frame)

    # Tahminlerden en yüksek olasılığa sahip sınıfı seçme
    predicted_label = np.argmax(predictions)

    # Güven eşiği kontrolü
    confidence_threshold = 0.5
    if predictions[0, predicted_label] > confidence_threshold:
        # Nesne sınıfını ve konumunu çizme
        label_text = f'Class: {predicted_label}'
        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Kutu içine alma işlemi
        h, w, _ = frame.shape
        box_color = (0, 255, 0)  # Kutu rengi (BGR formatında)
        box_thickness = 2  # Kutu kalınlığı

        cv2.rectangle(frame, (0, 0), (w, 50), box_color, -1)  # Arkaplan kutusu
        cv2.putText(frame, label_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)  # Sınıf etiketi

        cv2.imshow('Object Detection', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step


KeyboardInterrupt: 

In [10]:

import cv2
import numpy as np
import tensorflow as tf


# Open the video file
video_path = "Assignment/Training_Video_Jimmy Kimmel-Jake Johnson.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    # Read a frame from the video stream
    ret, frame = cap.read()

    if not ret:
        break

    # Preprocess the frame (resize if necessary)
    resized_frame = cv2.resize(frame, (64, 64))

    # Normalize the frame
    normalized_frame = resized_frame / 255.0

    # Reshape the frame to match the model's expected input shape
    input_frame = np.reshape(normalized_frame, (1, 64, 64, 3))

    # Use the model for prediction
    predictions = model.predict(input_frame)

    # Get the predicted class index
    predicted_class_index = np.argmax(predictions)

    # Get the class label using the LabelEncoder
    predicted_class_label = label_encoder.inverse_transform([predicted_class_index])[0]

    # Perform face detection using a face detection algorithm or a pre-trained face detector
    # In this example, we use the Haar Cascade classifier
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)

    # Draw bounding boxes around the detected faces and display the class labels
    for (x, y, w, h) in faces:
        # Draw the bounding box
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)

        # Display the predicted class label
        cv2.putText(frame, predicted_class_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)

    # Display the frame
    cv2.imshow('Video', frame)

    # Break the loop if 'q' key is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object and close all windows
cap.release()
cv2.destroyAllWindows()



1/1 [==============================] - 0s 18ms/step


KeyboardInterrupt: 

In [ ]:

import cv2
import numpy as np
import tensorflow as tf

# Video dosyasını oku
video_path = 'Assignment/Training_Video_Jimmy Kimmel-Jake Johnson.mp4'
cap = cv2.VideoCapture(video_path)

# MTCNN yüz algılama modelini yükle
detector = MTCNN()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Yüz tespiti
    faces = detector.detect_faces(frame)

    for face in faces:
        x, y, width, height = face['box']

        # Yüz bölgesini modelin giriş boyutuna uygun hale getir
        face_roi = cv2.resize(frame[y:y + height, x:x + width], (64, 64))
        normalized_face = face_roi / 255.0
        input_face = np.expand_dims(normalized_face, axis=0)

        # Yüzü tahmin et
        prediction = model.predict(input_face)
        predicted_class = np.argmax(prediction)
        label = str(predicted_class)

        # Yüzü çerçeve içine al
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, f'Predicted Class: {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Görüntüyü göster veya kaydet
    cv2.imshow('Object Detection', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()




In [ ]:
import cv2
import numpy as np
from mtcnn.mtcnn import MTCNN

# Load the trained model
model = tf.keras.models.load_model("my_model.h5")

# Define class names
class_names = ['celebrity1', 'celebrity2', 'celebrity3', 'celebrity4', 'celebrity5']

# Video dosyasını oku
video_path = 'Data/Training_Video_Jimmy Kimmel-Jake Johnson.mp4'
cap = cv2.VideoCapture(video_path)

# MTCNN yüz algılama modelini yükle
detector = MTCNN()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Yüz tespiti
    faces = detector.detect_faces(frame)

    for face in faces:
        x, y, width, height = face['box']

        # Yüz bölgesini modelin giriş boyutuna uygun hale getir
        face_roi = cv2.resize(frame[y:y + height, x:x + width], (64, 64))
        normalized_face = face_roi / 255.0
        input_face = np.expand_dims(normalized_face, axis=0)

        # Yüzü tahmin et
        prediction = model.predict(input_face)
        predicted_class = np.argmax(prediction)
        label = class_names[predicted_class]

        # Yüzü çerçeve içine al
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, f'Predicted Class: {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Görüntüyü göster veya kaydet
    cv2.imshow('Object Detection', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 19ms/step


In [ ]:

import cv2
import numpy as np
import tensorflow as tf

# Video dosyasını oku
video_path = 'Assignment/Training_Video_Jimmy Kimmel-Jake Johnson.mp4'
cap = cv2.VideoCapture(video_path)

# MTCNN yüz algılama modelini yükle
detector = MTCNN()

while cap.isOpened():
    ret, frame = cap.read()

    if not ret:
        break

    # Yüz tespiti
    faces = detector.detect_faces(frame)

    for face in faces:
        x, y, width, height = face['box']

        # Yüz bölgesini modelin giriş boyutuna uygun hale getir
        face_roi = cv2.resize(frame[y:y + height, x:x + width], (64, 64))
        normalized_face = face_roi / 255.0
        input_face = np.expand_dims(normalized_face, axis=0)

        # Yüzü tahmin et
        prediction = model.predict(input_face)
        predicted_class = np.argmax(prediction)
        label = str(predicted_class)

        # Yüzü çerçeve içine al
        cv2.rectangle(frame, (x, y), (x + width, y + height), (0, 255, 0), 2)
        cv2.putText(frame, f'Predicted Class: {label}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Görüntüyü göster veya kaydet
    cv2.imshow('Object Detection', frame)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()